# Team 88: Project Dataset Details
## Airport Traffic Data

In [2]:
#modules
import pandas as pd
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt 
%matplotlib inline

### [Airport and Airline Data](https://www.transtats.bts.gov/tables.asp?db_id=125&DB_Name=Airline%20Origin%20and%20Destination%20Survey%20(DB1B))
The Bureau of Transportation Statistics provides data on a 10% sample of flights and airline tickets going back to 1993 and recorded quarterly. The data can be filtered by destination airport and date. Bay Area destination airports of interest to us are San Francisco International Airport and Oakland International Airport. 

### Data Acquisition
With this data, the extraction process is a bit more manually involved as we need to download the files for each year and each quarter separately before combining them and making sure all the variables track before starting the analysis and exploration. There are two tables we need to explore in order to get all our variables of interest. These are `DB1BMarket` and `DB1BTicket`, and can be found [here](https://www.transtats.bts.gov/tables.asp?db_id=125&DB_Name=Airline%20Origin%20and%20Destination%20Survey%20%28DB1B%29). Part of our decision making will also include deciding how many years to explore - especially as we hope to be looking at the effects of wildfires on mobility and travel in the last decade. As this is only 10% of the actual traffic, we are hoping the sample is representative enough of trends.

Some relevant variables for us include for each of the tables includes:

***DB1BMarket***
- ItinID (int): a unique itinerary identifier which will help us connect the two tables together
- Orig and Dest (string): three letter airport codes for the origin/destination airport
- OriginAirportID(int) and DestAirportID (int): a unique identifier for the origin/destination airport
- Year and Quarter (string and int)
- Passengers (int): number of passengers

***DB1BTicket***
- ItinID (int): a unique itinerary identifier which will help us connect the two tables together
- RoundTrip (int): a round trip indicator (1=Yes, 0=No)
- ItinFare and FarePerMile (float): fares for the entire itinerary and per mile flown, respectively

### Dataset Structure and Features
To see more of the dataset, we are using a sample of the downloads for `2019 - Q1` for both `DB1BMarket` and `DB1BTicket`to see what variables look like and how we would think of using them for the purposes of this project.

In [5]:
df_market = pd.read_csv('data/920348143_T_DB1B_MARKET.csv')
df_ticket = pd.read_csv('data/920348143_T_DB1B_TICKET.csv')

In [6]:
#size of the data
for table, name in zip([df_market.shape, df_ticket.shape], ['market', 'ticket']):
    print('The airport', name, 'data has {} rows and {} columns'.format(table[0], table[1]))

The airport market data has 7141323 rows and 7 columns
The airport ticket data has 4390179 rows and 7 columns


In [7]:
#market view
df_market.head(2)

,ITIN_ID,ORIGIN_AIRPORT_ID,ORIGIN,DEST_AIRPORT_ID,DEST,PASSENGERS,Unnamed: 6
0,2019310,10135,ABE,10397,ATL,1.0,NaN
1,2019311,10135,ABE,10397,ATL,1.0,NaN


In [11]:
df_market.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7141323 entries, 0 to 7141322
Data columns (total 7 columns):
 #   Column             Dtype  
---  ------             -----  
 0   ITIN_ID            int64  
 1   ORIGIN_AIRPORT_ID  int64  
 2   ORIGIN             object 
 3   DEST_AIRPORT_ID    int64  
 4   DEST               object 
 5   PASSENGERS         float64
 6   Unnamed: 6         float64
dtypes: float64(2), int64(3), object(2)
memory usage: 381.4+ MB


In [8]:
#ticket view
df_ticket.head(2)

,ITIN_ID,YEAR,QUARTER,ROUNDTRIP,ITIN_YIELD,ITIN_FARE,Unnamed: 6
0,2019310,2019,3,0.0,0.0000,0.0,NaN
1,2019311,2019,3,0.0,0.0072,5.0,NaN


In [12]:
df_ticket.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4390179 entries, 0 to 4390178
Data columns (total 7 columns):
 #   Column      Dtype  
---  ------      -----  
 0   ITIN_ID     int64  
 1   YEAR        int64  
 2   QUARTER     int64  
 3   ROUNDTRIP   float64
 4   ITIN_YIELD  float64
 5   ITIN_FARE   float64
 6   Unnamed: 6  float64
dtypes: float64(4), int64(3)
memory usage: 234.5 MB


For each table, we have picked up an empty column that has no meaning so we can drop it for both to save computational time while processing the dataframes.

In [ ]:
#del df_market['Unnamed: 6'], df_ticket['Unnamed: 6']

In [10]:
#checking if itinerary ID is unique
df_market['ITIN_ID'].value_counts()

201935934420    10
201931763496    10
201933115014     8
201938036        8
201933154796     8
                ..
20193852642      1
201932991688     1
201932995786     1
20193762526      1
201934813207     1
Name: ITIN_ID, Length: 4390179, dtype: int64

From the above market table data, we see that the count of unique values is the same as the total rows in the ticket table data. This confirms that the `INIT_ID` field is one we can use to connect the two tables.

### Supplemental Information
We can use other data from a quick search to see if there are other airports and cities we should be looking at in the Bay Area. Additionally, we can get a list of all cities listed here and find the corresponding county and use that to connect with other datasets e.g. air quality data and fire incident data. This would help us attach other datasets to this one. For linking data we anticipate two variables:
- Time (quarter and year)
- Location (city, county, and state)